<a href="https://colab.research.google.com/github/ebtihalaziz/arbertmo/blob/main/Arabic_how_to_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!wget -c https://raw.githubusercontent.com/ebtihalaziz/Aurora/main/ArabicVoca.txt

--2021-04-14 18:24:00--  https://raw.githubusercontent.com/ebtihalaziz/Aurora/main/ArabicVoca.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3401894 (3.2M) [text/plain]
Saving to: ‘ArabicVoca.txt’

ArabicVoca.txt      100%[===================>]   3.24M  --.-KB/s    in 0.04s   

2021-04-14 18:24:01 (79.0 MB/s) - ‘ArabicVoca.txt’ saved [3401894/3401894]



In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ir7a9bkn
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ir7a9bkn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 21.5MB/s 
     |████████████████████████████████| 870kB 49.5MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2106756 sha256=9d5851600133bae21781992cf83ff06f222fcbe6d88c1133762341cd7ef91db1
  Stored in directory: /tmp/pip-ephem-wheel-cache-16sgb9e8/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=244d304cc8496428702dc891f314fc8a939bcbe1b367e48c48614cd76f5d8db0
  Stored i

In [ ]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer


paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 1min 1s, sys: 2.99 s, total: 1min 4s
Wall time: 35.7 s


In [ ]:
#!mkdir AraBERTo
tokenizer.save_model("drive/MyDrive/ArBERTo")

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
#tokenizer.encode("Mi estas Julien.")
tokenizer.encode("الجو جميل اليوم")

In [ ]:
#tokenizer.encode("Mi estas Julien.").tokens
tokenizer.encode("الجو جميل اليوم").tokens

In [ ]:
# Check that we have a GPU
!nvidia-smi

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

In [ ]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast
#from transformers import BertTokenizer

tokenizer = RobertaTokenizerFast.from_pretrained("./drive/MyDrive/AraBERTo")
#tokenizer = BertTokenizer.from_pretrained("./drive/MyDrive/AraBERTo", max_len=512)

In [ ]:
#from transformers import RobertaForMaskedLM
from transformers import BertForMaskedLM

model = BertForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 84 million parameters

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./ArabicVoca.txt",
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/AraBERTo",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./drive/MyDrive/AraBERTo")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="AraBERTo",
    tokenizer="AraBERTo"
)

In [ ]:
# The sun <mask>.
# =>

fill_mask("the sun <mask>.")

In [ ]:
fill_mask("This is the beginning of a beautiful <mask>.")

# This is the beginning of a beautiful <mask>.
# =>